# Emotion Detection Data Preparation

This notebook demonstrates the steps to prepare the GoEmotions dataset for a multi-label classification task.

 We will perform the following steps:
1. Group and sum emotion labels for each unique text.
2. Clean the dataset by removing conflicting neutral labels.
3. Retain only the top 3 emotion labels per text.
4. Stratify the dataset based on the labels.
5. Visualize the data before and after each step.


## Step 1: Load the Data
First, we load the GoEmotions dataset and inspect its structure.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datasets import load_dataset

dataFrame = load_dataset('go_emotions', 'raw')

print("Total data size: ", len(dataFrame["train"]))
df = dataFrame["train"].to_pandas()
df.to_parquet("go_emotions.parquet")
df

## Step 2: Group and Sum Emotion Labels
Next, we group the dataset by text and sum the emotion labels for each unique text.

In [ ]:
# Columns containing emotion labels
emotions = df.columns[9:]

# Group by text and sum the emotion columns
df_grouped = df.groupby('text', as_index=False)[emotions].sum()

# Display the first few rows of the grouped dataset
print("Grouped Dataset:")
print(df_grouped.head())
df_grouped

## Step 3: Remove Conflicting Neutral Labels
We remove the neutral label if it is set amongst other labels.

In [ ]:
emotions[:-1]

In [ ]:
# Clean up the dataset by removing the neutral label if it conflicts with other labels
def remove_conflicting_neutral(row):
    if row['neutral'] > 0 and row[emotions[:-1]].sum() > 0:
        row['neutral'] = 0
    return row

df_grouped = df_grouped.apply(remove_conflicting_neutral, axis=1)

# Display the first few rows after removing conflicting neutral labels
print("Dataset after Removing Conflicting Neutral Labels:")
display(df_grouped.head())
df_grouped

## Step 4: Retain Only Top 3 Labels
For each row, we keep only the top 3 emotion labels with the highest scores and set those to 1, while setting all other labels to 0.


In [ ]:
import numpy as np


# List of emotion columns
emotion_columns = df_grouped.columns.drop('text')

# Function to keep only the top 3 labels
def keep_top_3_labels(row):
    top_3_indices = row.nlargest(3).index
    row[:] = 0
    row[top_3_indices] = 1
    return row

# Apply the function to each row
df_grouped[emotion_columns] = df_grouped[emotion_columns].apply(keep_top_3_labels, axis=1)

# Display the first few rows after keeping only the top 3 labels
print("Dataset after Keeping Only Top 3 Labels:")
print(df_grouped.head())
df_grouped

## Step 5: Stratify the Dataset
We stratify the dataset based on the labels to ensure the distribution of labels is maintained.

In [ ]:
from sklearn.model_selection import train_test_split

# Create a multi-label indicator for stratification
df_grouped['labels'] = df_grouped[emotions].apply(lambda x: tuple(x), axis=1)

# Count the occurrences of each label combination
label_counts = df_grouped['labels'].value_counts()

# Identify rare combinations (those that appear less than twice)
rare_combinations = label_counts[label_counts < 2].index
print("Rare Combinations:", len(rare_combinations))

# Filter out rows with rare combinations
df_filtered = df_grouped[~df_grouped['labels'].isin(rare_combinations)]

# Stratify the dataset
train, test = train_test_split(df_filtered, test_size=0.2, stratify=df_filtered['labels'])

# Drop the 'labels' column as it was only needed for stratification
train = train.drop(columns=['labels'])
test = test.drop(columns=['labels'])

# Display the resulting datasets
print("Training set size:", len(train))
print("Test set size:", len(test))
print("First few rows of training set:\n", train.head())
print("First few rows of test set:\n", test.head())



## Visualizations
Let's visualize the distribution of labels before and after each step.

### Original Dataset


In [ ]:
# Plot the original distribution of emotion labels
import matplotlib.pyplot as plt
import seaborn as sns

original_emotion_counts = df[emotions].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(hue=original_emotion_counts.index, y=original_emotion_counts.values, palette="viridis")
plt.title('Original Counts per Emotion in Go Emotions Dataset')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


### Grouped Dataset

In [ ]:
# Plot the distribution of emotion labels after grouping
grouped_emotion_counts = df_grouped[emotions].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(hue=grouped_emotion_counts.index, y=grouped_emotion_counts.values, palette="viridis")
plt.title('Grouped Counts per Emotion in Go Emotions Dataset')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


### Cleaned Dataset

In [ ]:
# Plot the distribution of emotion labels after removing conflicting neutral labels
cleaned_emotion_counts = df_grouped[emotions].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(hue=cleaned_emotion_counts.index, y=cleaned_emotion_counts.values, palette="viridis")
plt.title('Cleaned Counts per Emotion in Go Emotions Dataset')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


### Dataset with Top 3 Labels


In [ ]:
# Plot the distribution of emotion labels after keeping only the top 3 labels
top_3_emotion_counts = df_grouped[emotions].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_3_emotion_counts.index, y=top_3_emotion_counts.values, palette="viridis")
plt.title('Counts per Emotion after keeping only the Top 3 Labels')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


### Stratified Training and Test Sets

In [ ]:
# Plot the distribution of emotion labels in the training set
train_emotion_counts = train[emotions].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x=train_emotion_counts.index, y=train_emotion_counts.values, palette="viridis")
plt.title('Training Set Counts per Emotion after Stratification')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


### Test Set


In [ ]:
# Plot the distribution of emotion labels in the test set
test_emotion_counts = test[emotions].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x=test_emotion_counts.index, y=test_emotion_counts.values, palette="viridis")
plt.title('Test Set Counts per Emotion after Stratification')
plt.xlabel('Emotion')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Calculate emotion counts for the training set
train_emotion_counts = train[emotions].sum()

# Calculate emotion counts for the test set
test_emotion_counts = test[emotions].sum()

# Combine the counts into a DataFrame
emotion_counts_df = pd.DataFrame({
    'Train': train_emotion_counts,
    'Test': test_emotion_counts
})

# Display the combined DataFrame
print("Emotion Counts in Train and Test Sets:")
print(emotion_counts_df)


In [ ]:
import matplotlib.pyplot as plt

# Plot the stacked bar chart
emotion_counts_df.plot(kind='bar', stacked=True, figsize=(12, 8), color=['skyblue', 'lightgreen'])

# Add titles and labels
plt.title('Stacked Distribution of Emotions in Train and Test Sets')
plt.xlabel('Emotions')
plt.ylabel('Counts')
plt.xticks(rotation=45)
plt.legend(title='Dataset')
plt.show()


## Step 6: Stacked Distribution of Emotions in Train and Test Sets
Finally, let's visualize the distribution of emotions in the training and test sets using a stacked bar chart.

In [ ]:
# Calculate emotion counts for the training set
train_emotion_counts = train[emotions].sum()

# Calculate emotion counts for the test set
test_emotion_counts = test[emotions].sum()

# Combine the counts into a DataFrame
emotion_counts_df = pd.DataFrame({
    'Train': train_emotion_counts,
    'Test': test_emotion_counts
})

# Calculate total counts for sorting
emotion_counts_df['Total'] = emotion_counts_df['Train'] + emotion_counts_df['Test']

# Sort the DataFrame by total counts
emotion_counts_df = emotion_counts_df.sort_values(by='Total', ascending=False)

# Drop the total column for plotting
emotion_counts_df = emotion_counts_df.drop(columns=['Total'])

# Display the sorted combined DataFrame
print("Sorted Emotion Counts in Train and Test Sets:")
print(emotion_counts_df)



In [ ]:
import matplotlib.pyplot as plt

# Plot the stacked bar chart
emotion_counts_df.plot(kind='bar', stacked=True, figsize=(12, 8), color=['skyblue', 'lightgreen'])

# Add titles and labels
plt.title('Stacked Distribution of Emotions in Train and Test Sets (Sorted)')
plt.xlabel('Emotions')
plt.ylabel('Counts')
plt.xticks(rotation=45)
plt.legend(title='Dataset')
plt.show()


## Save the datasets to Parquet file

In [ ]:
train.to_parquet("go_emotions_train.parquet")
test.to_parquet("go_emotions_test.parquet")